# Project: Youtube Analyzer
## Authors: Karmehr Arora, Alfred Koh

### Project Overview
Data: Youtube API’s
GUI: Tkinter

Roles:

    Karmehr: Accessing Data from the API & organizing it

    Alfred: Analyzing the data & representing it with tkinter

Goal: 
Finding the top 10 trending basketball videos (or videos of a requested topic) in the last 24 hours
Look for the most (total likes + total comments + total shares) under tag = basketball in the last 24 hours

Limitations: 
Estimated limit of searches of 50 per day due to Youtube API usage restrictions


In [3]:
# import utilized libraries pertaining to data collection
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [4]:
# get access key to API
# don't push to github, but hardcode for demo
from getpass import getpass
API_KEY = getpass('Enter your API Key:')

Enter your API Key:········


In [5]:
# define the api service and version
api_service_name = 'youtube'
api_version = 'v3'

## Requesting User to choose topic
default to basketball as topic if user has error/opts out of entering topic

In [64]:
# request user for topic (default = 'basketball')
topic = 'basketball'
topic = input()

basketball 2v2


## Retrieving current Datetime

In [65]:
from datetime import datetime
from datetime import timedelta

# datetime object containing current date and time
current_datetime = datetime.now() - timedelta(days=7)
print("datetime =", current_datetime)

# in accordance with Youtube Datetime rules; format =  YY/mm/ddTH:M:SZ
modified_dt = current_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")
print("date and time =", modified_dt)

datetime = 2023-11-22 00:55:03.375409
date and time = 2023-11-22T00:55:03Z


## Search youtube for videos with that tag

In [68]:
# access the youtube api client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_KEY)

# make a request to the api to find videos under that topic
# note: cost of this operation is 100 units of the alloted 10,000 units per 24 hours
request = youtube.search().list(
        part="id,snippet",
        maxResults=50,
        publishedAfter=modified_dt,
        q=topic
    )

In [69]:
# execute the query in the request
response = request.execute()

In [71]:
#   for(var i in results.items) {
#     var item = results.items[i];
#     Logger.log('[%s] Title: %s', item.id.videoId, item.snippet.title);
#   }
print("Search ID: ", response["items"][0]['id'])
print("\nSnippet Data:",response["items"][0]['snippet'])
videoIDs = []
videos = response["items"]
count = 1
for video in videos:
    if('videoId' in video['id']):
        print(count, ": https://www.youtube.com/watch?v=" + video['id']['videoId'])
        videoIDs.append(video['id']['videoId'])
        print(video['snippet']['publishedAt'])
        count = count+1
    else: 
        # this seems to only print out channels meaning the youtube search only returns videos & channels
        print(video['snippet'])

Search ID:  {'kind': 'youtube#video', 'videoId': 'hKllMekE7vU'}

Snippet Data: {'publishedAt': '2023-11-22T22:00:15Z', 'channelId': 'UCDSfK-fHq5T1DJl-d2DXyfg', 'title': 'Their BEEF Only Got WORSE After This 1v1… | Dreko vs Chauncey', 'description': 'Previously Chauncey & Dreko had faced off in a 1v1 to 13... Last time, Chauncey emerged victorious leaving Dreko waiting on the ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/hKllMekE7vU/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/hKllMekE7vU/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/hKllMekE7vU/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'The Next Chapter', 'liveBroadcastContent': 'none', 'publishTime': '2023-11-22T22:00:15Z'}
1 : https://www.youtube.com/watch?v=hKllMekE7vU
2023-11-22T22:00:15Z
2 : https://www.youtube.com/watch?v=C0jNRMv1JsI
2023-11-22T22:30:11Z
3 : https://www.youtube.com/watch?v=SRsF2sKfJzc
2023-11-22T

## Calculate and Find top 10 most trending videos under that tag

In [72]:
collection_of_video_data = {}
for video in videos:
    if('videoId' in video['id']):
        request = youtube.videos().list(part = 'snippet, statistics', id = video['id']['videoId'])
        response = request.execute()
        collection_of_video_data[video['id']['videoId']] = response

In [73]:
set_of_trend_totals = []
count = 1
for vid_data in collection_of_video_data:
#     print(count, ':', collection_of_video_data[vid_data])
    print("Statistics: ", collection_of_video_data[vid_data]['items'][0]['statistics'])
    
    trendTotal = 0
    if "viewCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        views = collection_of_video_data[vid_data]['items'][0]['statistics']['viewCount']
        trendTotal = trendTotal + int(views)
    if "likeCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        likes = collection_of_video_data[vid_data]['items'][0]['statistics']['likeCount']
        trendTotal = trendTotal + int(likes)
    if "commentCount" in collection_of_video_data[vid_data]['items'][0]['statistics']:
        comments = collection_of_video_data[vid_data]['items'][0]['statistics']['commentCount']
        trendTotal = trendTotal + int(comments)

    set_of_trend_totals.append(trendTotal)
    print(count, ': Trend Total: ', trendTotal)
    count = count+1

Statistics:  {'viewCount': '148666', 'likeCount': '8690', 'favoriteCount': '0', 'commentCount': '931'}
1 : Trend Total:  158287
Statistics:  {'viewCount': '55786', 'likeCount': '2161', 'favoriteCount': '0', 'commentCount': '66'}
2 : Trend Total:  58013
Statistics:  {'viewCount': '13993', 'likeCount': '451', 'favoriteCount': '0', 'commentCount': '111'}
3 : Trend Total:  14555
Statistics:  {'viewCount': '1', 'likeCount': '0', 'favoriteCount': '0', 'commentCount': '0'}
4 : Trend Total:  1
Statistics:  {'viewCount': '106746', 'likeCount': '3590', 'favoriteCount': '0', 'commentCount': '838'}
5 : Trend Total:  111174
Statistics:  {'viewCount': '60', 'favoriteCount': '0', 'commentCount': '0'}
6 : Trend Total:  60
Statistics:  {'viewCount': '733', 'likeCount': '19', 'favoriteCount': '0', 'commentCount': '5'}
7 : Trend Total:  757
Statistics:  {'viewCount': '2741', 'likeCount': '40', 'favoriteCount': '0', 'commentCount': '3'}
8 : Trend Total:  2784
Statistics:  {'viewCount': '89', 'likeCount': 

In [74]:
count = 0
video_id_to_trend_total = {}
for video in videoIDs:
    print('videoID: trendTotal --> ', video, end = ": ")
    print(set_of_trend_totals[count])
    video_id_to_trend_total[video] = set_of_trend_totals[count]
    count = count + 1
    
# for video in video_id_to_trend_total:
#     print(video, ":", video_id_to_trend_total[video])

videoID: trendTotal -->  hKllMekE7vU: 158287
videoID: trendTotal -->  C0jNRMv1JsI: 58013
videoID: trendTotal -->  SRsF2sKfJzc: 14555
videoID: trendTotal -->  AiMeSlnt8uY: 1
videoID: trendTotal -->  st1Ig_6-zHc: 111174
videoID: trendTotal -->  Xg_jQCWL0XI: 60
videoID: trendTotal -->  A3NGtT-yeKk: 757
videoID: trendTotal -->  mQgyemhvk3s: 2784
videoID: trendTotal -->  5XUTz2plOOk: 93
videoID: trendTotal -->  ohlvp58_z00: 647
videoID: trendTotal -->  yn8n9PM_ZnE: 1748
videoID: trendTotal -->  l71sq5hx0Xk: 13
videoID: trendTotal -->  1i12DmGGm_o: 19891
videoID: trendTotal -->  y_RXKagrlwM: 87
videoID: trendTotal -->  C2zSnZm7mPQ: 1759
videoID: trendTotal -->  4V6FPPO2G-4: 457
videoID: trendTotal -->  kMrz6MLlNUI: 28735
videoID: trendTotal -->  bKsoJdu--O4: 5216
videoID: trendTotal -->  IGv2tGjmx-c: 357
videoID: trendTotal -->  RpBfhOhWfu8: 2501
videoID: trendTotal -->  9n9FrI5Wqys: 2
videoID: trendTotal -->  9RjJEgxntuY: 11
videoID: trendTotal -->  RMZc0QChcX0: 70
videoID: trendTotal -->  